# Drone Vehicle Detection Analysis

This notebook demonstrates the vehicle detection and counting system for aerial drone footage.

## Contents
1. Setup and Configuration
2. Data Loading and Exploration
3. Model Testing
4. Detection Analysis
5. Tracking Analysis
6. Results Visualization
7. Performance Metrics

## 1. Setup and Configuration

In [ ]:
import sys
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

# Add src to path
sys.path.append('../src')

from src.detection.vehicle_detector import VehicleDetector
from src.tracking.multi_tracker import MultiTracker
from src.utils.config_manager import ConfigManager
from src.utils.video_processor import VideoProcessor
from src.visualization.result_visualizer import ResultVisualizer
from src.utils.statistics_manager import StatisticsManager

# Set up matplotlib for inline plotting
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# Load configuration
config_manager = ConfigManager('../config.yaml')
config = config_manager.get_config()

print("Configuration loaded:")
print(f"Model: {config['model']['type']}")
print(f"Target classes: {config['detection']['target_classes']}")
print(f"Confidence threshold: {config['model']['confidence_threshold']}")

## 2. Data Loading and Exploration

In [ ]:
# List available sample data
data_dir = Path('../data/sample')
if data_dir.exists():
    sample_files = list(data_dir.glob('*'))
    print(f"Found {len(sample_files)} sample files:")
    for file in sample_files:
        print(f"  - {file.name}")
else:
    print("No sample data directory found. Please add sample videos to ../data/sample/")
    print("You can use any drone footage with vehicles for testing.")

In [ ]:
# If you have a sample video, load it here
sample_video_path = None

# Uncomment and modify the path below if you have a sample video
# sample_video_path = '../data/sample/drone_footage.mp4'

if sample_video_path and os.path.exists(sample_video_path):
    video_processor = VideoProcessor(config)
    video_info = video_processor.get_video_info(sample_video_path)
    
    print("Video Information:")
    for key, value in video_info.items():
        print(f"  {key}: {value}")
else:
    print("No sample video loaded. You can still run detection on individual images.")

## 3. Model Testing

In [ ]:
# Initialize detector
detector = VehicleDetector(config)
print(f"Detector initialized with device: {detector.device}")
print(f"Model: {detector.model_config['type']}")

In [ ]:
# Test detection on a sample frame
if sample_video_path and os.path.exists(sample_video_path):
    # Read first frame from video
    cap = cv2.VideoCapture(sample_video_path)
    ret, test_frame = cap.read()
    cap.release()
    
    if ret:
        print(f"Test frame shape: {test_frame.shape}")
        
        # Run detection
        detections = detector.detect(test_frame)
        print(f"Found {len(detections)} detections")
        
        # Display detection stats
        if detections:
            stats = detector.get_detection_stats(detections)
            print("\nDetection Statistics:")
            print(f"  Total detections: {stats['total_count']}")
            print(f"  Class counts: {stats['class_counts']}")
            print(f"  Average confidence: {stats['avg_confidence']:.3f}")
        
        # Visualize results
        vis_frame = detector.visualize_detections(test_frame, detections)
        
        # Convert BGR to RGB for matplotlib
        vis_frame_rgb = cv2.cvtColor(vis_frame, cv2.COLOR_BGR2RGB)
        
        plt.figure(figsize=(15, 10))
        plt.imshow(vis_frame_rgb)
        plt.title(f'Vehicle Detection Results - {len(detections)} vehicles detected')
        plt.axis('off')
        plt.show()
    else:
        print("Could not read frame from video")
else:
    print("No sample video available for testing")
    print("You can test with a static image by loading it with cv2.imread()")

## 4. Detection Analysis

In [ ]:
# Analyze detection performance across multiple frames
if sample_video_path and os.path.exists(sample_video_path):
    frame_count = 0
    all_detections = []
    detection_counts = []
    confidences = []
    
    # Process first 100 frames for analysis
    for frame in video_processor.read_video(sample_video_path):
        if frame_count >= 100:  # Limit for notebook performance
            break
            
        detections = detector.detect(frame)
        all_detections.extend(detections)
        detection_counts.append(len(detections))
        
        if detections:
            frame_confidences = [d.confidence for d in detections]
            confidences.extend(frame_confidences)
        
        frame_count += 1
        
        if frame_count % 20 == 0:
            print(f"Processed {frame_count} frames...")
    
    print(f"\nAnalysis complete: {frame_count} frames processed")
    print(f"Total detections: {len(all_detections)}")
    print(f"Average detections per frame: {np.mean(detection_counts):.2f}")
    print(f"Average confidence: {np.mean(confidences):.3f}")
else:
    print("No sample video available for multi-frame analysis")

In [ ]:
# Visualize detection statistics
if 'detection_counts' in locals() and detection_counts:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Detection count over time
    axes[0, 0].plot(detection_counts)
    axes[0, 0].set_title('Detections per Frame')
    axes[0, 0].set_xlabel('Frame Number')
    axes[0, 0].set_ylabel('Number of Detections')
    
    # Detection count histogram
    axes[0, 1].hist(detection_counts, bins=20, alpha=0.7)
    axes[0, 1].set_title('Detection Count Distribution')
    axes[0, 1].set_xlabel('Number of Detections')
    axes[0, 1].set_ylabel('Frequency')
    
    # Confidence distribution
    if confidences:
        axes[1, 0].hist(confidences, bins=30, alpha=0.7)
        axes[1, 0].set_title('Confidence Score Distribution')
        axes[1, 0].set_xlabel('Confidence')
        axes[1, 0].set_ylabel('Frequency')
    
    # Class distribution
    if all_detections:
        class_counts = {}
        for detection in all_detections:
            class_counts[detection.class_name] = class_counts.get(detection.class_name, 0) + 1
        
        classes = list(class_counts.keys())
        counts = list(class_counts.values())
        
        axes[1, 1].pie(counts, labels=classes, autopct='%1.1f%%')
        axes[1, 1].set_title('Vehicle Class Distribution')
    
    plt.tight_layout()
    plt.show()

## 5. Tracking Analysis

In [ ]:
# Initialize tracker and visualizer
tracker = MultiTracker(config)
visualizer = ResultVisualizer(config)
stats_manager = StatisticsManager(config)

print("Tracker and visualizer initialized")

In [ ]:
# Run detection and tracking on sample frames
if sample_video_path and os.path.exists(sample_video_path):
    frame_count = 0
    sample_frames = []
    
    # Process frames with tracking
    for frame in video_processor.read_video(sample_video_path):
        if frame_count >= 50:  # Limit for notebook
            break
        
        # Detection
        detections = detector.detect(frame)
        
        # Tracking
        tracks = tracker.update(detections, frame)
        
        # Statistics
        stats_manager.update_frame_stats(frame_count, detections, tracks)
        
        # Save sample frames for visualization
        if frame_count % 10 == 0:  # Every 10th frame
            annotated_frame = visualizer.draw_annotations(frame, detections, tracks)
            sample_frames.append(annotated_frame)
        
        frame_count += 1
        
        if frame_count % 10 == 0:
            print(f"Processed {frame_count} frames with tracking...")
    
    print(f"\nTracking analysis complete: {frame_count} frames")
    
    # Get tracking statistics
    tracking_stats = tracker.get_tracking_statistics()
    print("\nTracking Statistics:")
    for key, value in tracking_stats.items():
        print(f"  {key}: {value}")
else:
    print("No sample video available for tracking analysis")

## 6. Results Visualization

In [ ]:
# Display sample tracked frames
if 'sample_frames' in locals() and sample_frames:
    n_frames = min(4, len(sample_frames))
    
    fig, axes = plt.subplots(2, 2, figsize=(20, 15))
    axes = axes.flatten()
    
    for i in range(n_frames):
        frame_rgb = cv2.cvtColor(sample_frames[i], cv2.COLOR_BGR2RGB)
        axes[i].imshow(frame_rgb)
        axes[i].set_title(f'Frame {i * 10} - Detection & Tracking')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("No sample frames available for visualization")

In [ ]:
# Create comprehensive statistics visualization
if 'stats_manager' in locals():
    final_stats = stats_manager.generate_final_report()
    
    print("Final Statistics Report:")
    print("=" * 50)
    
    if 'summary' in final_stats:
        summary = final_stats['summary']
        print(f"Frames processed: {summary.get('frames_processed', 0)}")
        print(f"Total detections: {summary.get('total_detections', 0)}")
        print(f"Unique tracks: {summary.get('unique_tracks', 0)}")
        print(f"Processing FPS: {summary.get('frames_per_second', 0):.2f}")
    
    if 'density_metrics' in final_stats:
        density = final_stats['density_metrics']
        print(f"\nDensity Metrics:")
        print(f"  Average density: {density.get('avg_density', 0):.2f}")
        print(f"  Max density: {density.get('max_density', 0)}")
        print(f"  Density trend: {density.get('density_trend', 'unknown')}")
    
    if 'class_distribution' in final_stats:
        class_dist = final_stats['class_distribution']
        print(f"\nClass Distribution:")
        if 'detections' in class_dist:
            for class_name, count in class_dist['detections'].items():
                print(f"  {class_name}: {count}")
else:
    print("No statistics available")

## 7. Performance Metrics

In [ ]:
# Analyze performance metrics
if 'final_stats' in locals() and 'performance_metrics' in final_stats:
    perf = final_stats['performance_metrics']
    
    print("Performance Analysis:")
    print("=" * 30)
    print(f"Average processing time: {perf.get('avg_processing_time', 0):.3f} seconds")
    print(f"Estimated FPS: {perf.get('fps_estimate', 0):.2f}")
    print(f"Max processing time: {perf.get('max_processing_time', 0):.3f} seconds")
    print(f"Min processing time: {perf.get('min_processing_time', 0):.3f} seconds")
    
    # Create performance visualization
    if 'stats_manager' in locals():
        processing_times = stats_manager.processing_times
        
        if processing_times:
            fig, axes = plt.subplots(1, 2, figsize=(15, 5))
            
            # Processing time over frames
            axes[0].plot(processing_times)
            axes[0].set_title('Processing Time per Frame')
            axes[0].set_xlabel('Frame Number')
            axes[0].set_ylabel('Processing Time (seconds)')
            
            # Processing time histogram
            axes[1].hist(processing_times, bins=20, alpha=0.7)
            axes[1].set_title('Processing Time Distribution')
            axes[1].set_xlabel('Processing Time (seconds)')
            axes[1].set_ylabel('Frequency')
            
            plt.tight_layout()
            plt.show()
else:
    print("No performance metrics available")

## Conclusion

This notebook demonstrates the complete pipeline for drone vehicle detection and counting:

1. **Detection**: Using YOLO models to identify vehicles in aerial footage
2. **Tracking**: Maintaining object identities across frames
3. **Analysis**: Computing traffic density and flow metrics
4. **Visualization**: Creating annotated outputs and statistical plots

### Next Steps:
- Add your own drone footage to `../data/sample/`
- Experiment with different model configurations
- Analyze longer video sequences
- Implement custom traffic analysis features

### Key Features:
- Real-time processing capabilities
- Multiple vehicle class detection
- Robust multi-object tracking
- Comprehensive statistics and visualization
- Configurable parameters for different scenarios